# Мультиклассовая классификация корпуса текстов с дообучением модели BERT на основе тематического моделирования

## Подготовка среды

### Установка библиотек

#### Для работы с данными в датасете и обучения модели используется библиотека Transformers (Hugging Face)

In [ ]:
! pip install -q transformers datasets
! pip install -U accelerate
! pip install -U transformers
! pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 8.0 MB/s eta 0:00:00


#### Подключение к репозиторию Hugging Face для последующего сохранения модели

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-c

In [ ]:
notebook_login()

## Подготовка данных

In [ ]:
import numpy as np
import pandas as pd

#### Для разметки подготовлен корпус новостных статей с сайта, посвященного информационной безопасности (https://www.securitylab.ru/). По результатам работы над промежуточным курсовым проектом корпус был размечен с помощью тематического моделирования (LDA). Каждому тексту были присвоены от одной до трех пометок с указанием содержащейся в тексте темы.

In [ ]:
# загрузка данных с Google Drive
from google.colab import drive
drive.mount('/content/drive')
SecurityLab = pd.read_csv("drive/MyDrive/Colab Notebooks/Magellan-Text-Navigator/data/Secure_df_processed.csv", encoding = "UTF-8", sep = ",")
full_texts = list(SecurityLab["Text"])

Mounted at /content/drive


#### Пример данных из датасета

In [ ]:
SecurityLab[:5]

,Unnamed: 0,Date,Title,Subtitle,Original_tags,Topic_tag_1,Topic_tag_2,Topic_tag_3,Name_tags,Keywords,Text
0,https://www.securitylab.ru/news/536258.php,2023-02-02,Администрация США: «В iOS должна появиться уст...,В отчете Белого дома критикуется политика мага...,"Google, боковая загрузка, NTIA, Apple, Евросою...",программное_обеспечение,госрегулирование,кибербезопасность,"Google, Европа, Нидерланды, Samsung, Apple, Ев...","apple, европа, приложение, store, система, and...",Корпорации Apple и Google уже давно сталкивают...
1,https://www.securitylab.ru/news/536256.php,2023-02-02,Северокорейские хакеры воспользовались уязвимо...,И причём тут вообще ананас?,"Cisco Talos, КНДР, Северная Корея, GREASE, Dtr...",вредоносы,уязвимости,финансы_и_бизнес,"Zimbra, Кндр, Kimsuky","dtrack, zimbra, сервер, pineapple, уязвимость,...",Северокорейская группировка Lazarus Group испо...
2,https://www.securitylab.ru/news/536257.php,2023-02-02,Лаборатория Касперского: разработчики и пентес...,Эксперты рассказали - кого ищут подпольные раб...,"хакер, Лаборатория Касперского, пентест, разра...",события_и_мероприятия,программное_обеспечение,законы_и_нарушения,NaN,"лаборатория, касперский, зарплата, согласно, о...","Согласно отчёту Лаборатории Касперского, разра..."
3,https://www.securitylab.ru/news/536253.php,2023-02-02,Евросоюз: почти 40% интернет-магазинов использ...,"Если вы думаете, что вас не проведешь - вы уже...","Норвегия, Евросоюз, интернет-магазин, Еврокоми...",госрегулирование,разработка,программное_обеспечение,"Еврокомиссия, Норвегия, Евросоюз","паттерн, манипулирование, право, защита, еврос...","Евросоюз заявил, что почти 40% интернет-магази..."
4,https://www.securitylab.ru/news/536252.php,2023-02-02,"Встречайте LockBit Green: шифровальщик новый, ...",«Позеленевший» вымогатель данных разработан на...,"LockBit Green, VX-Underground, шифровальщик, C...",мошенничества,вредоносы,NaN,"Conti, Lockbit","conti, lockbit, green, исходный, шифровальщик,...",Банда вымогателей LockBit снова начала использ...


### Форматирование датасета

#### Для удобства работы с данными датасет приводится к формату DatasetDict библиотеки Datasets. Полученный объект будет содержать три отдельных датасета для обучения, валидации и тестирования модели.

In [ ]:
from datasets import Dataset
from datasets import DatasetDict

In [ ]:
# выделение данных из датасета

ID = list(SecurityLab.index)
Text = list(SecurityLab["Text"])
labels = list(set(SecurityLab["Topic_tag_1"]))

In [ ]:
# пометки из соответствующих колонок группируются в один список

topic_list = []
for i in range(len(SecurityLab)):
  temp = []
  temp_str = ""
  for a in range(1,4):
    if type(SecurityLab["Topic_tag_{0}".format(str(a))][i]) == str:
      temp.append(SecurityLab["Topic_tag_{0}".format(str(a))][i])
    temp_str = ", ".join(temp)
  topic_list.append(temp_str)
SecurityLab["all_topics"] = topic_list

#### Пометки приводятся к булевому формату: каждой пометке (будущему классу) соответствует отдельная колонка со значениями TRUE или FALSE.


In [ ]:
for i in labels:
  SecurityLab['{0}'.format(str(i))] = SecurityLab["all_topics"].map(lambda x: str(i) in x)

#### Данные собираются в словарь, который затем преобразуется в объект DatasetDict, содержащий три отдельных датасета для обучения, валидации и тестирования модели.

In [ ]:
# форматирование словаря

SecurityDict = {"ID": ID, 'Text': list(SecurityLab["Text"])}
for i in labels:
  SecurityDict['{0}'.format(str(i))] = SecurityLab['{0}'.format(str(i))]

In [ ]:
# форматирование датасета

SecurityDS = Dataset.from_dict(SecurityDict)
train_testvalid = SecurityDS.train_test_split(test_size = 0.25)
test_valid = train_testvalid['test'].train_test_split(test_size = 0.5)
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

#### Для работы с пометками (лейблами) датасета создаются кодированные списки

In [ ]:
# форматирование кодированных списков для работы с пометками (лейблами) датасета

labels = [label for label in dataset['train'].features.keys() if label not in ['ID', 'Text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['законы_и_нарушения',
 'разработка',
 'исследования',
 'вредоносы',
 'уязвимости',
 'утечки',
 'мошенничества',
 'госрегулирование',
 'программное_обеспечение',
 'финансы_и_бизнес',
 'события_и_мероприятия',
 'кибербезопасность']

#### Демонстрация датасета

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'Text', 'законы_и_нарушения', 'разработка', 'исследования', 'вредоносы', 'уязвимости', 'утечки', 'мошенничества', 'госрегулирование', 'программное_обеспечение', 'финансы_и_бизнес', 'события_и_мероприятия', 'кибербезопасность'],
        num_rows: 6006
    })
    test: Dataset({
        features: ['ID', 'Text', 'законы_и_нарушения', 'разработка', 'исследования', 'вредоносы', 'уязвимости', 'утечки', 'мошенничества', 'госрегулирование', 'программное_обеспечение', 'финансы_и_бизнес', 'события_и_мероприятия', 'кибербезопасность'],
        num_rows: 1002
    })
    validation: Dataset({
        features: ['ID', 'Text', 'законы_и_нарушения', 'разработка', 'исследования', 'вредоносы', 'уязвимости', 'утечки', 'мошенничества', 'госрегулирование', 'программное_обеспечение', 'финансы_и_бизнес', 'события_и_мероприятия', 'кибербезопасность'],
        num_rows: 1001
    })
})

In [ ]:
example = dataset['train'][0]
example

{'ID': 3846,
 'Text': 'В Сети был опубликован PoC-код для эксплуатации уязвимости удаленного выполнения кода в VMware (CVE-2022-22954), которая уже используется в реальных хакерских атаках. Преступники в ходе атак устанавливают на системы жертв майнеры криптовалют. Уязвимость получила оценку в 9,8 балла из максимальных 10 по шкале CVSS и затрагивает популярные программные продукты VMware Workspace ONE Access и VMware Identity Manager. 6 апреля 2022 года поставщик программного обеспечения выпустил уведомление касательно данной уязвимости, а также обновление безопасности, устраняющее проблему. Вскоре многие исследователи в области кибербезопасности разработали работающие эксплойты для CVE-2022-22954, причем по крайней мере один PoC-код для эксплуатации уязвимости был опубликован в Twitter. Специалисты компании Bad Packets обнаружили попытки использовать уязвимость в реальных атаках. В ходе одной из таких атак использовался бэкдор Tsunami для Linux-систем. По словам исследователя в област

## Предобработка данных

#### Для классификации текста с помощью BERT необходимо токенизировать данные с помощью токенизатора из библиотеки Transformers. Для предобработки данных, как и для дальнейшей классификации используется дообученная на русскоязычных текстах модель BERT От DeepPavlov.

In [ ]:
import os
import json

In [ ]:
# загрузка модели из репозитория

!gdown http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz
!tar -xzf /content/rubert_cased_L-12_H-768_A-12_pt.tar.gz

Downloading...
From: http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz
To: /content/rubert_cased_L-12_H-768_A-12_pt.tar.gz
100% 662M/662M [00:33<00:00, 19.5MB/s]


In [ ]:
# подготовка модели к работе

with open("/content/rubert_cased_L-12_H-768_A-12_pt/bert_config.json", "r") as read_file, open("/content/rubert_cased_L-12_H-768_A-12_pt/config.json", "w") as conf:
    file = json.load(read_file)
    conf.write(json.dumps(file))
!rm /content/rubert_cased_L-12_H-768_A-12_pt/bert_config.json

In [ ]:
# подготовка токенизатора и соответствующей функции

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("rubert_cased_L-12_H-768_A-12_pt")

def preprocess_data(examples):
  # выделение текстов из датасета
  text = examples["Text"]
  # кодирование текстов
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=256)
  # добавление пометок
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # создается матрица (размер партии данных, количество пометок)
  labels_matrix = np.zeros((len(text), len(labels)))
  # заполнение матрицы
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]
  encoding["labels"] = labels_matrix.tolist()

  return encoding

In [ ]:
# применение функции токенизации

encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/6006 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [ ]:
# приведение данных к формату тензора PyTorch

encoded_dataset.set_format("torch")

#### Пример предобработанных данных

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
tokenizer.decode(example['input_ids'])

'[CLS] в сети был опубликован poc - код для эксплуатации уязвимости удаленного выполнения кода в vmware ( cve - 2022 - 22954 ), которая уже используется в реальных хакерских атаках. преступники в ходе атак устанавливают на системы жертв маинеры криптовалют. уязвимость получила оценку в 9, 8 балла из максимальных 10 по шкале cvss и затрагивает популярные программные продукты vmware workspace one access и vmware identity manager. 6 апреля 2022 года поставщик программного обеспечения выпустил уведомление касательно даннои уязвимости, а также обновление безопасности, устраняющее проблему. вскоре многие исследователи в области кибербезопасности разработали работающие эксплоиты для cve - 2022 - 22954, причем по краинеи мере один poc - код для эксплуатации уязвимости был опубликован в twitter. специалисты компании bad packets обнаружили попытки использовать уязвимость в реальных атаках. в ходе однои из таких атак использовался бэкдор tsunami для linux - систем. по словам исследователя в облас

In [ ]:
example['labels']

tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.])

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['уязвимости', 'финансы_и_бизнес']

## Обучение модели

#### Определение модели для классификации

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("rubert_cased_L-12_H-768_A-12_pt",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at rubert_cased_L-12_H-768_A-12_pt were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

#### Установка параметров

In [ ]:
import accelerate
import transformers
# проверка версий
transformers.__version__, accelerate.__version__
from transformers import TrainingArguments, Trainer

In [ ]:
# переменные для передачи в модель

batch_size = 8
metric_name = "f1"

In [ ]:
# параметры модели

args = TrainingArguments(
    f"bert-eval-256",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

#### Определение функции для вычисления метрик

#### Задается набор метрик для оценки дообученной модели и сравнения версий. В дальнейшем результаты сохраняются в репозиторий вместе с моделью.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

def multi_label_metrics(predictions, labels, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

#### Проверка данных и параметров обучения

In [ ]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [ ]:
encoded_dataset['train']['input_ids'][0]

tensor([   101,    845,  13148,   2067,  12559,  11026,    237,    130,  10044,
          2748,  21023,  85374,  69052,   1766,  21366,  31416,    845,    274,
           257,  27956,    120,    236,  11124,    130,  73190,    130,  26728,
         36695,    122,    128,   5754,   4745,  12404,    845,  33788,  90276,
          2999,  86923,    132,  36119,    845,   7887,  10196,  53503,   1469,
          8513,  13212,  21984,  91808,    880, 102424,    132,  82762,  10395,
         22902,    845,    154,    128,    152,  30280,   1703, 104485,   3955,
          1516,  40217,    236,    275,  12540,    851,  67574,  38595,  79878,
         24860,    274,    257,  27956,  11936, 109863,  10976,  14133,  49953,
           851,    274,    257,  27956,  13116,  22200,  11707,  11329,  37759,
           132,    148,   6167,  73190,   1768,  87796,  29268,  15881,  19994,
         63400,  67192,   3813,   1438,    852,  85374,    128,    625,   3060,
         42832,  10050,    128,  20374, 

In [ ]:
# передача информации в нейронной сети, forward pass

outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7285, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.1467,  0.2071, -0.0173, -0.1554,  0.1381,  0.1613,  0.5635, -0.4061,
          0.4611, -0.0708,  0.0125,  0.0297]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

#### Дообучение модели

In [ ]:
# создание объекта Trainer для дообучения и применения модели

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/AndrewDOrlov/bert-eval-256 into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.40k/679M [00:00<?, ?B/s]

Download file runs/Jul05_15-46-14_0268970a1d22/events.out.tfevents.1688572363.0268970a1d22.1391.0: 100%|######…

Download file training_args.bin: 100%|##########| 3.87k/3.87k [00:00<?, ?B/s]

Download file runs/Jul05_12-27-26_95c02c3db7e3/events.out.tfevents.1688560094.95c02c3db7e3.2635.0: 100%|######…

...

Download file runs/Jul05_12-27-26_95c02c3db7e3/events.out.tfevents.1688562207.95c02c3db7e3.2635.1: 100%|######…

Clean file training_args.bin:  26%|##5       | 1.00k/3.87k [00:00<?, ?B/s]

Clean file runs/Jul05_12-27-26_95c02c3db7e3/events.out.tfevents.1688560094.95c02c3db7e3.2635.0:  10%|#        …

Clean file runs/Jul05_12-27-26_95c02c3db7e3/events.out.tfevents.1688562207.95c02c3db7e3.2635.1: 100%|#########…

Clean file pytorch_model.bin:   0%|          | 1.00k/679M [00:00<?, ?B/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.406500,0.304723,0.673446,0.765625,0.176823
2,0.258500,0.267691,0.733305,0.808948,0.255744
3,0.207200,0.250413,0.753975,0.826129,0.293706
4,0.158700,0.244895,0.766850,0.839025,0.312687
5,0.133500,0.245772,0.771143,0.841444,0.326673


TrainOutput(global_step=3755, training_loss=0.22843155029134332, metrics={'train_runtime': 1979.6894, 'train_samples_per_second': 15.169, 'train_steps_per_second': 1.897, 'total_flos': 3950967205785600.0, 'train_loss': 0.22843155029134332, 'epoch': 5.0})

## Оценка модели

In [ ]:
trainer.evaluate()

{'eval_loss': 0.2605232298374176,
 'eval_f1': 0.7521578298397041,
 'eval_roc_auc': 0.8283391283535946,
 'eval_accuracy': 0.3006993006993007,
 'eval_runtime': 16.1669,
 'eval_samples_per_second': 61.917,
 'eval_steps_per_second': 7.794,
 'epoch': 5.0}

## Тестирование модели

In [ ]:
trainer.eval_dataset=encoded_dataset["test"]

In [ ]:
trainer.evaluate()

{'eval_loss': 0.24590303003787994,
 'eval_f1': 0.7657472210786331,
 'eval_roc_auc': 0.8392317829770035,
 'eval_accuracy': 0.30538922155688625,
 'eval_runtime': 16.8526,
 'eval_samples_per_second': 59.457,
 'eval_steps_per_second': 7.477,
 'epoch': 5.0}

## Сохранение модели


In [ ]:
# загрузка модели в репозиторий

trainer.push_to_hub("bert-eval-256")

Upload file runs/Jul05_12-27-26_95c02c3db7e3/events.out.tfevents.1688560094.95c02c3db7e3.2635.0:   0%|        …

Upload file runs/Jul05_12-27-26_95c02c3db7e3/events.out.tfevents.1688562207.95c02c3db7e3.2635.1:   0%|        …

To https://huggingface.co/AndrewDOrlov/bert-eval-256
   bab0a3a..8637082  main -> main

   bab0a3a..8637082  main -> main

To https://huggingface.co/AndrewDOrlov/bert-eval-256
   8637082..12b4b81  main -> main

   8637082..12b4b81  main -> main



'https://huggingface.co/AndrewDOrlov/bert-eval-256/commit/8637082827157e9030d16304038454c873bedbf6'

In [ ]:
# сохранение в локальное хранилище
# trainer.save_model("/content/CL_1")

## Применение модели к неразмеченным данным

In [ ]:
SecurityLabTest = pd.read_csv("drive/MyDrive/Colab Notebooks/Magellan-Text-Navigator/data/SecurityLabTest.csv", encoding = "UTF-8", sep = ",")
texts = list(SecurityLabTest["Text"])

In [ ]:
texts[0]

'Исследование показало, что использование искусственного интеллекта в разработке программного обеспечения может существенно повысить производительность и инновации в мире.Разработчики программного обеспечения сталкиваются с постоянно растущим спросом на качественные и сложные приложения, которые могут решать различные задачи и потребности. Однако создание такого программного обеспечения требует много времени, усилий и знаний. Чтобы упростить и ускорить этот процесс, компания GitHub, являющаяся крупнейшим в мире хранилищем для разработчиков программного обеспечения, представила менее года назад новый инструмент под названием GitHub Copilot. Это расширение для Visual Studio и других инструментов разработчика, которое работает на базе OpenAI Codex - одной из самых продвинутых технологий генеративного искусственного интеллекта (ИИ). GitHub Copilot помогает создавать программы, предлагая скрипты, предложения на естественном языке и автозавершение кода. Новые версии инструмента также интегри

#### Очистка кэша

In [ ]:
torch.cuda.empty_cache()

#### Применение модели

In [ ]:
len(texts[0:50] 50:100 100 : 150 150 : 200)

3

In [ ]:
outputs_list=[]
for text in texts[:50]:
  encoding = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=256)
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
  outputs = trainer.model(**encoding)
  outputs_list.append(outputs)

In [ ]:
logits_list=[]
for i in outputs_list:
  logits = i.logits
  logits_list.append(logits)
logits.shape

torch.Size([1, 12])

In [ ]:
label_list = []
for logit in logits_list:
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(logit.squeeze().cpu())
  predictions = np.zeros(probs.shape)
  predictions[np.where(probs >= 0.5)] = 1
  predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
  label_list.append(predicted_labels)

## Сохранение датасета

In [ ]:
#подготовка списков для датасета

original_tags = list(SecurityLabTest["Tags"][:50])
dates = list(SecurityLabTest["Date"][:50])
texts = list(SecurityLabTest["Text"][:50])
titles = list(SecurityLabTest["Title"][:50])
subtitles = list(SecurityLabTest["Subtitle"][:50])
news_links = list(SecurityLabTest["Unnamed: 0"][:50])
topic_tags = list([i for i in label_list])

In [ ]:
#сохранение датасета

data_list = dates, titles, subtitles, original_tags, topic_tags, texts
SecurityLabTest_df = pd.DataFrame(data_list, columns = news_links).T
SecurityLabTest_df = SecurityLabTest_df.rename(columns={0: "Date", 1:"Title", 2:"Subtitle", 3:"Original_tags", 4:"Topic_tags", 5:"Text"})
SecurityLabTest_df.to_csv("drive/MyDrive/Colab Notebooks/Magellan-Text-Navigator/data/SVD_Secure_df_Test_256_1.csv", encoding="utf-8")